### Chapter 10

이 장은 9장에서 만든 Vector2d 클래스를 개선하면서 진행될 것이다. 목표는 Vector 클래스가 기본적인 시퀀스 프로토콜과 슬라이싱, 집합 해싱, 포맷 언어 확장을 지원하도록 하는 것이다.

+ 다차원 벡터 클래스

먼저 n차원 벡터를 지원하는, 다차원 벡터 클래스를 만들어 보자.

In [2]:
from array import array
import reprlib #요소가 너무 많을 때 ...을 사용해 축약해주는 기능을 위해
import math

class Vector:
  typecode='d'

  def __init__(self, components):
    self._components=array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components=reprlib.repr(self._components)
    components=components[components.find('['):-1] # [] 바깥의 글자들은 지워준다
    return 'Vector({})'.format(components)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))

  def __eq__(self, other): #비교 연산자 오버로딩
    return tuple(self)==tuple(other)

  def __abs__(self): #다차원 벡터의 크기
    return math.sqrt(sum(x*x for x in self))

  def __bool__(self):
    return bool(abs(self))

  @classmethod
  def frombytes(cls, octets):
    typecode=chr(octets[0])
    memv=memoryview(octets[1:]).cast(typecode)
    return cls(memv)


+ `__repr__` 관리

9장에서 만든, 여러 기능을 지원하는 벡터 클래스를 다차원으로 확장했다. 그런데 repr 특별 메서드의 경우, 차원이 많은 벡터를 한번에 출력하면 디버깅 등이 쉽지 않으므로, reprlib 라이브러리를 사용하여 출력 문자열의 길이를 제한하도록 했다. 이렇게 하면 대형 구조체도 안전하게 표현할 수 있다.

또한 Vector 클래스는 내부적으로 array 로 관리되는데, 우리는 repr의 출력을 array 형태 대신 [a,b,c] 형태로 간략하게 만들고 싶었다. 그것을 위해서 repr 특별 메서드 내부에서 [] 바깥의 글자들을 전부 지워주는 방식을 택했다(정확히는 슬라이싱을 이용해 [] 내부 글자들만 취하는 방식)

이때 주의할 점은, repr은 디버깅에 사용되므로 절대 예외를 발생시키면 안 된다는 점이다. repr 구현 내에서 무언가 잘못될 경우, 문제를 내부적으로 해결하고 사용자가 객체를 알아볼 수 있도록 최상의 형태를 출력하는 식으로 코드를 짜야 한다.

+ 시퀀스 프로토콜 구현

아무튼 이제 원래의 목적이었던, Vector 클래스가 시퀀스 프로토콜을 구현하도록 코드를 짜 보자.

파이썬에서는 완전히 작동하는 시퀀스형을 만들기 위해 특별한 클래스를 상속할 필요가 없다. 파이썬은 덕 타이핑으로 돌아가므로, 그저 시퀀스 프로토콜에 따르는 메서드를 구현하기만 하면 된다. 파이썬의 시퀀스 프로토콜은 `__len__`과 `__getitem__` 특별 메서드에 의해서 돌아가므로, 이 메서드만 구현해 주면 되는 것이다.

또 프로토콜은 비공식적이고, 강제로 적용되는 사항이 아니므로(문서에는 정의되어 있지만, 실제 코드에서 이를 정의하지는 않는다) 클래스가 사용되는 환경에 따라 일부만 구현할 수도 있다. 예를 들어 반복을 지원하려면 `__getitem__` 특별 메서드만 구현하면 되고 `__len__`은 구현할 필요 없다.

In [3]:
class Test:
  def __init__(self, numList):
    self._nums=numList

  def __getitem__(self, pos):
    return self._nums[pos]

n=Test([1,2,3,4,5])
for i in n:
  print(i)

1
2
3
4
5


위의 클래스는 다른 특별 메서드는 아무것도 구현되지 않았고 오로지 `__getitem__`만 구현되었는데 정상적으로 반복을 수행하는 걸 볼 수 있다. 아무튼 이제 정말로 Vector클래스의 시퀀스 프로토콜을 만들자. len과 getitem 을 구현하면 우리가 만든 Vector클래스에도 슬라이싱을 적용할 수 있다. 함수 객체를 이용해서 바로 Vector 클래스의 특별 메서드를 추가해 주자.

In [4]:
def length(vector):
  return len(vector._components)

def VectorGetItem(vector, idx):
  return vector._components[idx]

Vector.__len__=length
Vector.__getitem__=VectorGetItem

v1=Vector([3,4,5,6,7])
len(v1)
print(v1[0], v1[1])
print(v1[1:4])

3.0 4.0
array('d', [4.0, 5.0, 6.0])


인덱싱과 슬라이싱이 정상적으로 지원되는 걸 볼 수 있다. 그런데 리스트의 슬라이싱도 리스트가 되는 것처럼, Vector 클래스의 슬라이싱도 Vector가 되면 좋겠다. 현재는 Vector의 슬라이싱 결과물은 array 가 된다.

+ 더 나은 슬라이싱

이런 기능을 위해서는 슬라이싱 연산을 배열에 위임하면 안 되고 `__getitem__` 메서드를 제대로 만들어야 한다. 먼저 슬라이싱의 작동 방식을 보자. 그것을 보기 위해 먼저, 전달받은 인수를 그대로 반환하는 `__getitem__` 메서드를 만든다.

In [ ]:
class MySeq:
  def __getitem__(self, index):
    return index

s=MySeq()
print(s[1])

print(s[1:4])

print(s[1:4:2])

print(s[1:4:2, 6:10])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), slice(6, 10, None))


getitem 메서드에 슬라이싱을 전달하면 그대로의 슬라이싱 객체가 반환되는 것을 볼 수 있다. 또한 [] 안에 콤마가 들어가면 `__getitem__` 이 슬라이싱 객체의 튜플을 반환하는 것도 볼 수 있다.

이번에는 슬라이스 객체에 대해 알아보기 위해서 슬라이스 클래스의 속성을 알아보자.

In [ ]:
print(dir(slice))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


슬라이스 객체가 가지고 있는 속성 중에 `indices` 함수가 있는 것을 볼 수 있다. 이 메서드는 무슨 역할을 하는 것일까? help를 이용하여 보면 `S.indices(len)` 은 길이가 len인 시퀀스 S가 나타내는 슬라이스의 start, stop 인덱스와 step 길이를 계산한다. 또한 경계를 벗어난 인덱스나 음수 인덱스도 매끄럽게 처리하는 기능을 담당한다. 즉 주어진 길이의 시퀀스 안에서 슬라이싱을 수행하도록 만들어 주는 역할을 하는 것이다. 작동 방식을 보면 이해가 빠르다.

In [ ]:
print(slice(None, 10, 2).indices(5))
print(slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


위 코드는 길이가 5인 시퀀스에 slice[:10:2]를 적용한 것과 [-3:] 을 적용한 것의 예시이다. 만약에 길이 5인 시퀀스에 [:10:2] 슬라이싱을 적용하면 [0:5:2] 슬라이싱을 적용한 것과 같은 결과가 나온다는 것이다.

물론 우리가 구현할 Vector 클래스에서는 slice 인수를 받아서 _components 배열에 슬라이싱 처리를 위임할 것이다. 하지만 기반 시퀀스 자료형에 슬라이싱 처리를 위임할 수 없을 때에는 이 indices 메서드가 큰 도움이 된다.

그럼 이제 Vector 클래스의 getitem 이 슬라이싱도 제대로 처리하도록 구현해 보자.

In [5]:
import numbers

def VectorGetItem2(vector, idx):
  cls=type(vector)
  if isinstance(idx, slice): #슬라이싱
    return cls(vector._components[idx])
  elif isinstance(idx, numbers.Integral): #인덱싱
    return vector._components[idx]
  else: #주어진 인수가 정수형/슬라이스 아닐 시 에러
    msg='{cls.__name__} indices must be integers' #에러 메세지
    raise TypeError(msg.format(cls=cls))

Vector.__getitem__=VectorGetItem2

In [6]:
v7=Vector(range(8))
print(v7[-1])
print(v7[1:4])

7.0
(1.0, 2.0, 3.0)


이제 우리는 벡터의 슬라이싱도 벡터가 되는 것을 확인했다. 또한 만약 벡터의 슬라이싱에 정수나 슬라이스 객체가 들어가지 않으면 에러가 발생할 것임을 알 수 있다.(그렇게 구현했으니까)

+ 동적 속성 접근

v.x나 v.y와 같이 벡터 요소를 이름으로 접근할 수 있는 기능을 만들 수는 없을까? 물론 property 데커레이터를 이용해서 x,y,z 등에 읽기 전용 접근을 만들 수도 있다. 하지만 이렇게 비슷한 프로퍼티를 여러 개 만드는 건 그다지 좋은 방식이 아니다. `__getattr__` 특별 메서드를 이용하면 훨씬 깔끔하게 구현 가능하다.

원리를 이해하기 위해 파이썬에서 객체의 속성을 탐색하는 방법에 대해 알아본다. 이를테면 obj.x 를 호출한다고 하자. 그러면 파이썬 인터프리터는 먼저 obj 객체에 x속성이 있는지를 검사한다. 그 속성이 없으면 객체의 클래스(`obj.__class__`) 에서 더 검색한다. 그리고 나서 클래스 상속 그래프를 따라 계속 올라간다(이 과정은 4부에서 자세히 설명한다) 만약 그래도 x 속성을 찾지 못하면, self와 속성명을 문자열로 전달하여 obj 클래스의 getattr 특별 메서드를 호출한다.

그럼 우리가 생각하는 대로의 getattr 메서드를 이제 구현해 보자. 지금까지 한 것처럼, 함수 객체를 이용해서 특별 메서드를 새로 넣어 준다.

In [ ]:
Vector.shortcut_names='xyzt'

def GetAttr(self, name):
  cls=type(self)
  if len(name)==1: #name이 만약 한 글자라면 shortcut_names에서 그 위치를 찾은 후 그 위치의 배열 항목을 반환.
    pos=cls.shortcut_names.find(name)
    if 0<=pos<len(self._components):
      return self._components[pos]
  msg='{.__name__!r} object has no attribute {!r}' #주어진 조건을 만족 못 하는 attribute일 경우 Error 발생
  raise AttributeError(msg.format(cls,name))

Vector.__getattr__=GetAttr

v=Vector(range(5))
print(v)
print(v.x)
print(v.z)

v.x=10 #원래 Vector 인스턴스에는 x 항목 같은 건 없다. 하지만 정상적으로 할당이 된다!
print(v.x)

(0.0, 1.0, 2.0, 3.0, 4.0)
0.0
2.0
10


그런데 문제가 생겼다. v.x 나 v.z 따위로 적절한 인덱스에 접근하는 것까지는 정상적으로 작동한다. 그런데 v.x에 할당을 한번 한 후 v.x 를 호출하면 왜 v[0] 이 아니라 할당한 값이 호출된다.

이건 `__getattr__` 특별 메서드는 해당 속성을 찾지 못했을 때 최후에 호출되는 수단이기 떄문이다. 만약에 v.x=10 같은 선언으로 v.x에 값이 할당되면 v객체에 자동으로 x 속성이 할당되므로 더 이상 v.x값을 가져오기 위해 `__getattr__` 메서드가 호출되지 않는다. 이미 v 객체 내에 있는 x 속성을 가져오면 되기 때문이다.

이런 불일치 문제를 해결하려면 Vector 클래스에서 속성값을 설정하는 부분을 수정해야 하고, 이는 setattr 특별 메서드를 수정하는 것으로 할 수 있다.

In [7]:
def SetAttr(self, name, value):
  cls=type(self)
  if len(name)==1:
    if name in cls.shortcut_names: #name이 xyzt 중에 있으면 구체적인 에러 발생
      error='readonly attribute {attr_name!r}'
    elif name.islower(): #name 이 그 외 소문자면 단일 문자 속성명에 대한 에러 메시지 설정
      error="can't set attributes 'a' to 'z' in {cls_name!r}"
    else:
      error=''
    if error: #error가 빈 문자열이 아니면 에러를 발생시킨다
      msg=error.format(cls_name=cls.__name__, attr_name=name)
      raise AttributeError(msg)
  super(self.__class__, self).__setattr__(name, value) #에러가 발생하지 않을 떄는 표준 동작을 위해 슈퍼클래스의 setattr을 호출

  #위 함수는 Vector클래스 코드에 직접 추가되는 것이 아니라 함수 객체를 새로 추가해 주는 것이므로 단순한 super() 만 쓰는 게 아니라
  #self의 클래스를 인수로 넣어서 슈퍼클래스를 명시적으로 호출해 준다.

Vector.__setattr__=SetAttr

위의 코드처럼 Vector의 `__setattr__` 특별 메서드를 짜 놓고 아까처럼 v.x=10과 같은 할당을 시도하면 에러가 발생하는 것을 볼 수 있을 것이다.

여기서 눈여겨볼 점은, 모든 속성의 설정을 막는 것이 아니라 지원되는 읽기 전용 속성 x,y,z,t와의 혼동을 피하기 위해, 단일 소문자로 되어 있는 속성의 설정만 막고 있다는 것이다.

그리고 9장에서 배웠던 `__slots__` 설정을 사용하고 싶은 유혹을 느낄 수도 있다. slots를 설정하면 설정된 속성 외의 다른 속성을 설정하는 것을 막을 수 있기 때문이다. 하지만 9.8절에서 설명한 것처럼, 단지 객체 속성의 새로운 생성을 막기 위해 slots를 사용하는 것은 좋은 선택이 아니다. 그것은 신중하게 생각한 후 메모리 절약을 위해 꼭 필요할 때만 사용해야 한다.

이 예제에서 가르쳐주는 것 또 하나는, 객체 동작의 불일치를 피하려면 `__getattr__` 구현시에 `__setattr__`도 함께 구현해야 한다는 것이다.

벡터 요소의 변경을 허용하고 싶은 경우도 있을 수 있는데, 이 경우 `__setitem__` 특별 메서드 구현 시에 v[0]=1.1 형태로 변경할 수 있게 되고, `__setattr__` 특별 메서드 구현 시에는 v.x=1.1형태로 요소를 변경할 수 있게 된다. 하지만 우리는 Vector를 해시 가능하게 만들고자 하므로 일단은 Vector 클래스를 불변형으로 유지한다.

+ 벡터 클래스 해싱

이제 우리가 만든 다차원 벡터 클래스를 해싱하는 메서드를 만들어 보자. 우리는 eq 특별 메서드를 이미 구현했으므로 hash 특별 메서드만 구현하면 Vector를 해시할 수 있게 된다.

9장에서 이미 xor을 이용해 해시하는 것을 보았으므로 다차원 벡터에서도 이를 적용해 보자. 이번에는 벡터의 요소가 여러 개 있을 수 있으므로, 모든 요소에 연속해서 xor을 적용한 것을 벡터 인스턴스의 해시값으로 하자. 이런 연산에는 functools.reduce 함수가 딱 맞다. reduce 함수는 all, any, sum 등으로 대체될 수 있어 그리 인기있지는 않지만 딱 맞는 경우가 있는 것이다.

먼저 reduce 함수의 사용을 보자.

In [8]:
import functools
print(functools.reduce(lambda a,b:a*b, range(1,6)))

120


reduce는 첫 번째 인수로는 함수 객체를 받고 두 번째 인수로는 반복형을 받는다. 그리고 첫 번째 인수인 함수에 반복형의 원소들을 차례로 적용하여, 결국은 하나의 값을 리턴하는 것이다. 반복형의 원소들의 전체 합이라든지, 곱이라든지.. 위 코드는 1에서 5까지 들어 있는 반복형을 받아 그 모든 값을 곱한 값을 리턴하는 코드인 것이다.

이 함수를 이용하면 우리는 벡터의 원소들에 차례로 해시를 적용할 수 있다! 그리고 람다 함수를 굳이 사용할 것 없이, operator 모듈에서는 xor을 포함한 모든 파이썬 중위 연산자를 함수 형태로 제공한다. 이것을 이용하자.

In [11]:
import operator

def VectorHash(self):
  hashes=(hash(x) for x in self._components)
  return functools.reduce(operator.xor, hashes, 0)
  #reduce의 초깃값도 지정해 주자

Vector.__hash__=VectorHash

v1=Vector([1,2,3,4,5])
v2=Vector([1.1,1.2,1.3])
print(hash(v1))
print(hash(v2))

1
922337203685476353


혹은, 위에서는 지능형 튜플을 사용해서 벡터 요소들의 해시값을 계산해 주었지만 map을 사용하면 모든 요소의 해시값을 계산한다는 걸 더 명시적으로 드러낼 수도 있을 것이다.

이번에는 우리가 만든 벡터 클래스의 eq 메서드를 살펴보자. 우리가 원래 구현한 건 Vector의 요소들을 복사한 튜플을 만든 후 그것을 비교하는 것이다. 하지만 많은 요소들을 가진 벡터의 경우, 그것들을 전부 복사해서 튜플을 두 개나 새로 만드는 건 비효율적이다. 벡터의 길이를 먼저 비교해서 길이가 다른 벡터는 당연히 다르다고 판정하고, 또 그 후에도 좀더 효율적으로 비교할 수 있는 방법이 없을까?

In [12]:
def VecEq(self, other):
  if len(self)!=len(other):
    return False
  for a,b in zip(self,other):
    if a!=b:return False
  return True

Vector.__eq__=VecEq
v1=Vector([1,2,3,4,5])
v2=Vector([1,2,3,4,5])
print(v1==v2)
print(v1 is v2)

True
False


위와 같이 코드를 짜면 두 벡터의 길이를 먼저 비교할 뿐더러, 벡터의 길이가 같아서 요소마다 비교해야 할 경우에도 zip을 이용해 제너레이터를 만들어 딱 필요한 만큼만 비교한다. 물론 all을 이용해서, 위의 for문조차 한 줄로 줄여버리는 것도 가능하다.

이때 쓰인 zip 함수는, 요청에 따라 각 반복형에서 나온 항목들을 튜플로 묶어 생성하는 제너레이터를 반환해 준다. 주의할 점은, zip함수는 인수로 받은 반복형 중 가장 길이가 짧은 것의 끝에 다다르면 종료된다. 따라서 위의 `__eq__` 메서드 오버로딩에서도 벡터의 크기를 비교해 주어야 한다. 그렇지 않으면 길이부터가 다른 벡터가 같다고 판정될 수 있기 때문이다. 그것만 유의한다면 zip 함수는 병렬 입력을 매우 쉽게 만들어 준다! 비슷하게는 enumerate 함수가 있다.

In [16]:
# zip 함수의 예시
l1=[10,20,30]
s=['ten', 'twenty', 'thirty']
z=zip(l1,s) # zip 함수는 제너레이터를 반환한다
for i in z:
  print(i)

(10, 'ten')
(20, 'twenty')
(30, 'thirty')
